# Building a Controller for Autonomous Car

This notebook teaches you how to build controllers for autonomous vehicles using the Simple Autonomous Car SDK.

## What You'll Learn

1. Understanding the control architecture
2. Using built-in controllers (Pure Pursuit, PID)
3. Building custom controllers
4. Integrating controllers with planners
5. Adding sensors to the car

## Architecture Overview

```
Planner → Plan (waypoints) → Controller → Control Commands → Car
         ↑                                                    ↓
         └────────────────── Sensors ─────────────────────────┘
```

The flow:
- **Planner**: Generates a path (plan) to follow
- **Controller**: Computes control commands to follow the plan
- **Car**: Executes the control commands
- **Sensors**: Provide perception data (can be used by planner/controller)

In [ ]:
import sys

sys.path.insert(0, '../../src')

import matplotlib.pyplot as plt
import numpy as np

from simple_autonomous_car import (
    Car,
    CarState,
    GroundTruthMap,
    LiDARSensor,
    PerceivedMap,
    PIDController,
    PurePursuitController,
    Track,
    TrackPlanner,
)

## Step 1: Setup - Create Car with Sensors

In [ ]:
# Create track
track = Track.create_simple_track(length=80.0, width=40.0, track_width=5.0)

# Create car at starting position
start_point, start_heading = track.get_point_at_distance(0.0)
car = Car(
    initial_state=CarState(
        x=start_point[0],
        y=start_point[1],
        heading=start_heading,
        velocity=8.0
    )
)

print(f"✓ Car created at ({car.state.x:.2f}, {car.state.y:.2f})")
print(f"  Initial velocity: {car.state.velocity:.2f} m/s")

### Adding Sensors to the Car

In [ ]:
# Create maps for sensors
ground_truth_map = GroundTruthMap(track)
perceived_map = PerceivedMap(
    ground_truth_map,
    position_noise_std=0.15,
    orientation_noise_std=0.08,
    measurement_noise_std=0.2,
)

# Create LiDAR sensor
lidar = LiDARSensor(
    ground_truth_map=ground_truth_map,
    perceived_map=perceived_map,
    max_range=40.0,
    angular_resolution=0.1,
    point_noise_std=0.1,
    name="front_lidar",
    pose_ego=np.array([0.0, 0.0, 0.0]),  # At car origin
)

# Add sensor to car
car.add_sensor(lidar)

print(f"✓ Added {len(car.sensors)} sensor(s) to car")
print(f"  Sensor: {car.sensors[0].name} (range: {car.sensors[0].max_range}m)")

# You can add more sensors!
# rear_lidar = LiDARSensor(..., name="rear_lidar", pose_ego=np.array([-1.0, 0.0, np.pi]))
# car.add_sensor(rear_lidar)

## Step 2: Creating a Planner

In [ ]:
# Create planner that follows the track
planner = TrackPlanner(
    track=track,
    lookahead_distance=50.0,  # Plan 50m ahead
    waypoint_spacing=2.0,     # Waypoints every 2m
    name="track_follower",
)

print("✓ Planner created")
print(f"  Lookahead: {planner.lookahead_distance}m")
print(f"  Waypoint spacing: {planner.waypoint_spacing}m")

# Generate a plan
plan = planner.plan(car.state)
print(f"\n✓ Generated plan with {len(plan)} waypoints")

### Visualization: The Plan

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

# Plot track
ax.plot(track.inner_bound[:, 0], track.inner_bound[:, 1], 'k-', linewidth=2, label='Track Boundaries')
ax.plot(track.outer_bound[:, 0], track.outer_bound[:, 1], 'k-', linewidth=2)
ax.plot(track.centerline[:, 0], track.centerline[:, 1], 'b--', linewidth=1.5, alpha=0.5, label='Centerline')

# Plot plan
if len(plan) > 0:
    ax.plot(plan[:, 0], plan[:, 1], 'g-o', linewidth=2, markersize=6, label='Plan (Waypoints)', alpha=0.7)

# Plot car
car_corners = car.get_corners()
from matplotlib.patches import Polygon

car_poly = Polygon(car_corners, closed=True, color='blue', alpha=0.7, label='Car')
ax.add_patch(car_poly)

ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_title('Planned Path')
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

print("\nVisualization shows:")
print("  - Black lines: Track boundaries")
print("  - Blue dashed: Track centerline")
print("  - Green line with dots: Generated plan (waypoints)")
print("  - Blue rectangle: Car position")

## Step 3: Creating a Controller

In [ ]:
# Create Pure Pursuit controller
controller = PurePursuitController(
    lookahead_distance=8.0,      # Look 8m ahead
    lookahead_gain=2.0,           # Adaptive lookahead gain
    max_steering_rate=1.0,        # Max steering rate (rad/s)
    target_velocity=10.0,         # Target velocity (m/s)
    velocity_gain=0.5,            # Velocity control gain
    name="pure_pursuit",
)

print("✓ Controller created")
print("  Type: Pure Pursuit")
print(f"  Lookahead: {controller.lookahead_distance}m")
print(f"  Target velocity: {controller.target_velocity} m/s")

## Step 4: Using Controller with Plan

In [ ]:
# Get perception data from sensors
perception_data = car.sense_all(environment_data={"ground_truth_map": ground_truth_map})

# Compute control commands using the plan
control = controller.compute_control(
    car_state=car.state,
    perception_data=perception_data,
    plan=plan,
    dt=0.1,
)

print("✓ Control commands computed:")
print(f"  Acceleration: {control['acceleration']:.2f} m/s²")
print(f"  Steering rate: {control['steering_rate']:.2f} rad/s")
print("\n  These commands will make the car follow the plan!")

## Step 5: Complete Control Loop

In [ ]:
# Complete control loop: Planner → Controller → Car
dt = 0.1
num_steps = 200

# Visualization setup
fig, ax = plt.subplots(figsize=(12, 10))
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_title('Car Following Plan with Controller')
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)

# Plot track
ax.plot(track.inner_bound[:, 0], track.inner_bound[:, 1], 'k-', linewidth=2)
ax.plot(track.outer_bound[:, 0], track.outer_bound[:, 1], 'k-', linewidth=2)

# Store car trajectory
trajectory = []

for step in range(num_steps):
    # 1. Generate/update plan
    plan = planner.plan(car.state)

    # 2. Get perception from sensors
    perception_data = car.sense_all(environment_data={"ground_truth_map": ground_truth_map})

    # 3. Compute control
    control = controller.compute_control(
        car_state=car.state,
        perception_data=perception_data,
        plan=plan,
        dt=dt,
    )

    # 4. Apply control to car
    car.update(
        dt=dt,
        acceleration=control["acceleration"],
        steering_rate=control["steering_rate"],
    )

    # Store trajectory
    trajectory.append(car.state.position().copy())

    # Update visualization every 10 steps
    if step % 10 == 0:
        ax.clear()
        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')
        ax.set_title(f'Step {step}: Car Following Plan')
        ax.set_aspect('equal')
        ax.grid(True, alpha=0.3)

        # Plot track
        ax.plot(track.inner_bound[:, 0], track.inner_bound[:, 1], 'k-', linewidth=2)
        ax.plot(track.outer_bound[:, 0], track.outer_bound[:, 1], 'k-', linewidth=2)

        # Plot plan
        if len(plan) > 0:
            ax.plot(plan[:, 0], plan[:, 1], 'g-o', linewidth=1.5, markersize=4, alpha=0.6, label='Plan')

        # Plot trajectory
        if len(trajectory) > 1:
            traj_array = np.array(trajectory)
            ax.plot(traj_array[:, 0], traj_array[:, 1], 'b-', linewidth=1.5, alpha=0.5, label='Trajectory')

        # Plot car
        car_corners = car.get_corners()
        car_poly = Polygon(car_corners, closed=True, color='blue', alpha=0.7)
        ax.add_patch(car_poly)

        ax.legend()
        plt.pause(0.01)

print(f"\n✓ Simulation complete! Car traveled {len(trajectory)} steps")
print(f"  Final position: ({car.state.x:.2f}, {car.state.y:.2f})")
print(f"  Final velocity: {car.state.velocity:.2f} m/s")

## Step 6: Building a Custom Controller

In [ ]:
from simple_autonomous_car.control.base_controller import BaseController


class MyCustomController(BaseController):
    """
    Example custom controller.

    This shows how to build your own controller by extending BaseController.
    """

    def __init__(self, target_velocity=10.0, name="custom"):
        super().__init__(name=name)
        self.target_velocity = target_velocity

    def compute_control(
        self,
        car_state: CarState,
        perception_data=None,
        plan=None,
        dt=0.1,
    ) -> dict:
        """
        Your custom control logic here!

        You can:
        - Use perception_data from sensors
        - Follow the plan
        - Implement your own control algorithm
        """
        if not self.enabled or plan is None or len(plan) == 0:
            return {"acceleration": 0.0, "steering_rate": 0.0}

        # Example: Simple heading control
        car_pos = car_state.position()
        distances = np.linalg.norm(plan - car_pos, axis=1)
        closest_idx = np.argmin(distances)

        if closest_idx < len(plan) - 1:
            target_point = plan[closest_idx + 1]  # Look one waypoint ahead
            target_vector = target_point - car_pos
            target_angle = np.arctan2(target_vector[1], target_vector[0])

            angle_error = target_angle - car_state.heading
            angle_error = np.arctan2(np.sin(angle_error), np.cos(angle_error))

            # Simple proportional control
            steering_rate = 2.0 * angle_error
            steering_rate = np.clip(steering_rate, -1.0, 1.0)
        else:
            steering_rate = 0.0

        # Velocity control
        velocity_error = self.target_velocity - car_state.velocity
        acceleration = 0.5 * velocity_error
        acceleration = np.clip(acceleration, -2.0, 2.0)

        return {"acceleration": acceleration, "steering_rate": steering_rate}

# Test custom controller
custom_controller = MyCustomController(target_velocity=12.0)
control = custom_controller.compute_control(car.state, plan=plan)

print("✓ Custom controller created and tested!")
print(f"  Acceleration: {control['acceleration']:.2f} m/s²")
print(f"  Steering rate: {control['steering_rate']:.2f} rad/s")

## Step 7: Comparing Different Controllers

In [ ]:
# Create different controllers
pure_pursuit = PurePursuitController(
    lookahead_distance=8.0,
    target_velocity=10.0,
    name="pure_pursuit",
)

pid_controller = PIDController(
    kp=1.5,
    ki=0.1,
    kd=0.2,
    target_velocity=10.0,
    name="pid",
)

controllers = [pure_pursuit, pid_controller]
controller_names = ["Pure Pursuit", "PID"]

# Test each controller
print("Comparing controllers:")
for controller, name in zip(controllers, controller_names):
    control = controller.compute_control(car.state, plan=plan)
    print(f"\n{name}:")
    print(f"  Acceleration: {control['acceleration']:.2f} m/s²")
    print(f"  Steering rate: {control['steering_rate']:.2f} rad/s")

## Summary

You've learned:

1. ✅ **Adding sensors to car**: Use `car.add_sensor()` to add LiDAR or other sensors
2. ✅ **Creating planners**: Use `TrackPlanner` or build custom planners
3. ✅ **Creating controllers**: Use built-in controllers or extend `BaseController`
4. ✅ **Control loop**: Planner → Controller → Car
5. ✅ **Modular architecture**: Everything is pluggable and extensible!

### Key Concepts

- **Sensors**: Modular, can add multiple sensors to a car
- **Planners**: Generate paths (plans) to follow
- **Controllers**: Compute control commands to follow plans
- **Car**: Executes control commands, manages sensors

### Next Steps

- Build custom sensors (Camera, Radar, etc.)
- Build custom planners (A*, RRT, etc.)
- Build custom controllers (MPC, LQR, etc.)
- Combine multiple sensors, planners, and controllers